In [105]:
#import statements
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

### Data Loading

Data sourced from https://www.kaggle.com/datasets/nikdavis/steam-store-games?select=steamspy_tag_data.csv

The author of the dataset documented their process of using Steam APIs here:
https://nik-davis.github.io/posts/2019/steam-data-collection/

The dataset is stored under a data folder in my local repository. 

The filepath is ./data/Steam_store_data

There are 6 CSV files in the dataset.

In [2]:
!ls

README.md
data
notebook.ipynb
scratch_notebook.ipynb


In [3]:
steam_df = pd.read_csv('./data/Steam_store_data/steam.csv')

In [4]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

### Exploring the data

In [5]:
steam_df.shape

(27075, 18)

There are 27,705 rows - each representing a game in Steam's store - and 18 columns.

Let's go through each column one-by-one:

In [6]:
#unique id value identifier for game
steam_df['appid']

0             10
1             20
2             30
3             40
4             50
          ...   
27070    1065230
27071    1065570
27072    1065650
27073    1066700
27074    1069460
Name: appid, Length: 27075, dtype: int64

In [8]:
#name of game
steam_df['name']

0                    Counter-Strike
1             Team Fortress Classic
2                     Day of Defeat
3                Deathmatch Classic
4         Half-Life: Opposing Force
                    ...            
27070               Room of Pandora
27071                     Cyber Gun
27072              Super Star Blast
27073    New Yankee 7: Deer Hunters
27074                     Rune Lord
Name: name, Length: 27075, dtype: object

In [9]:
#date of release (YY-MM-DD)
steam_df['release_date']

0        2000-11-01
1        1999-04-01
2        2003-05-01
3        2001-06-01
4        1999-11-01
            ...    
27070    2019-04-24
27071    2019-04-23
27072    2019-04-24
27073    2019-04-17
27074    2019-04-24
Name: release_date, Length: 27075, dtype: object

In [10]:
#release_date is stored as a string
type(steam_df['release_date'][0])

str

In [11]:
steam_df['english']

0        1
1        1
2        1
3        1
4        1
        ..
27070    1
27071    1
27072    1
27073    1
27074    1
Name: english, Length: 27075, dtype: int64

In [12]:
steam_df['english'].value_counts()

1    26564
0      511
Name: english, dtype: int64

There are 26,564 English games and 511 non-English games.

In [13]:
#name of game's developer
steam_df['developer']

0                     Valve
1                     Valve
2                     Valve
3                     Valve
4          Gearbox Software
                ...        
27070           SHEN JIAWEI
27071        Semyon Maximov
27072           EntwicklerX
27073    Yustas Game Studio
27074      Adept Studios GD
Name: developer, Length: 27075, dtype: object

In [14]:
#name of game's publisher
steam_df['publisher']

0                       Valve
1                       Valve
2                       Valve
3                       Valve
4                       Valve
                 ...         
27070             SHEN JIAWEI
27071        BekkerDev Studio
27072             EntwicklerX
27073    Alawar Entertainment
27074    Alawar Entertainment
Name: publisher, Length: 27075, dtype: object

In [15]:
#platform availability for game
steam_df['platforms']

0        windows;mac;linux
1        windows;mac;linux
2        windows;mac;linux
3        windows;mac;linux
4        windows;mac;linux
               ...        
27070              windows
27071              windows
27072              windows
27073          windows;mac
27074          windows;mac
Name: platforms, Length: 27075, dtype: object

In [16]:
steam_df['platforms'].value_counts()

windows              18398
windows;mac;linux     4623
windows;mac           3439
windows;linux          610
mac                      3
mac;linux                1
linux                    1
Name: platforms, dtype: int64

In [17]:
steam_df[steam_df['platforms'] == 'mac']

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
1413,214630,Call of Duty: Black Ops - Mac Edition,2012-09-27,1,Aspyr,Aspyr,mac,18,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Action;Zombies;Multiplayer,68,168,105,0,0,50000-100000,15.49
12479,569050,Paul Pixel - The Awakening,2017-01-09,1,Xoron GmbH,Xoron GmbH,mac,0,Single-player,Adventure;Indie,Adventure;Indie;Point & Click,0,5,0,0,0,0-20000,2.89
16662,694180,MobileZombie,2017-10-13,1,YIMING ZHANG,YIMING ZHANG,mac,0,Single-player;Partial Controller Support,Adventure;Casual;Free to Play;Indie,Free to Play;Adventure;Indie,0,14,11,0,0,0-20000,0.00


In [18]:
#age-requirement for game
steam_df['required_age']

0        0
1        0
2        0
3        0
4        0
        ..
27070    0
27071    0
27072    0
27073    0
27074    0
Name: required_age, Length: 27075, dtype: int64

In [22]:
steam_df['required_age'].value_counts()

0     26479
18      308
16      192
12       73
7        12
3        11
Name: required_age, dtype: int64

In [23]:
steam_df['categories']

0        Multi-player;Online Multi-Player;Local Multi-P...
1        Multi-player;Online Multi-Player;Local Multi-P...
2                    Multi-player;Valve Anti-Cheat enabled
3        Multi-player;Online Multi-Player;Local Multi-P...
4        Single-player;Multi-player;Valve Anti-Cheat en...
                               ...                        
27070                     Single-player;Steam Achievements
27071                                        Single-player
27072    Single-player;Multi-player;Co-op;Shared/Split ...
27073                            Single-player;Steam Cloud
27074                            Single-player;Steam Cloud
Name: categories, Length: 27075, dtype: object

In [21]:
#there are a lot of combinations for categories
#might have to clean this up separately to see pull out the most meaningful categories
steam_df['categories'].value_counts()

Single-player                                                                                                                                                                        6110
Single-player;Steam Achievements                                                                                                                                                     2334
Single-player;Steam Achievements;Steam Trading Cards                                                                                                                                  848
Single-player;Partial Controller Support                                                                                                                                              804
Single-player;Steam Trading Cards                                                                                                                                                     792
                                                                      

In [43]:
def extract_category(df, column):
    '''
    Returns a dictionary containing categories and their counts from the dataframe
    Categories and genres are stored separated by a semicolon
    '''
    category_dict = {}
    for tags in df[column]:
        split_tags = tags.split(';')
        for x in split_tags:
            if x not in category_dict:
                category_dict[x] = 1
            else:
                category_dict[x] += 1
    return category_dict

In [95]:
categories = extract_category(steam_df, 'categories')
categories

{'Multi-player': 3974,
 'Online Multi-Player': 2487,
 'Local Multi-Player': 1615,
 'Valve Anti-Cheat enabled': 94,
 'Single-player': 25678,
 'Steam Cloud': 7219,
 'Steam Achievements': 14130,
 'Steam Trading Cards': 7918,
 'Captions available': 721,
 'Partial Controller Support': 4234,
 'Includes Source SDK': 35,
 'Cross-Platform Multiplayer': 1081,
 'Stats': 1878,
 'Commentary available': 144,
 'Includes level editor': 1036,
 'Steam Workshop': 897,
 'In-App Purchases': 690,
 'Co-op': 1721,
 'Full controller support': 5695,
 'Steam Leaderboards': 3439,
 'SteamVR Collectibles': 40,
 'Online Co-op': 1071,
 'Shared/Split Screen': 2152,
 'Local Co-op': 1059,
 'MMO': 421,
 'VR Support': 231,
 'Mods': 2,
 'Mods (require HL2)': 1,
 'Steam Turn Notifications': 63}

In [97]:
#maybe subset categories based on number of players
categories_players = ['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Single-player', 'Co-op', 'Online Co-op', 'Shared/Split Screen', 'Local Co-op']
subset_categories = [x for x in categories if x in categories_players]
subset_categories

['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Single-player',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op']

In [6]:
steam_df['genres'].value_counts()

Action;Indie                                           1852
Casual;Indie                                           1482
Action;Adventure;Indie                                 1229
Adventure;Indie                                        1170
Action;Casual;Indie                                    1004
                                                       ... 
Violent;Action;Casual;Indie;Simulation;Early Access       1
Casual;Racing;Early Access                                1
Adventure;Indie;Simulation;Sports                         1
Massively Multiplayer;Early Access                        1
Utilities;Web Publishing                                  1
Name: genres, Length: 1552, dtype: int64

In [47]:
genre_dict = extract_category(steam_df, 'genres')
genre_dict

{'Action': 11903,
 'Free to Play': 1704,
 'Strategy': 5247,
 'Adventure': 10032,
 'Indie': 19421,
 'RPG': 4311,
 'Animation & Modeling': 79,
 'Video Production': 38,
 'Casual': 10210,
 'Simulation': 5194,
 'Racing': 1024,
 'Violent': 843,
 'Massively Multiplayer': 723,
 'Nudity': 266,
 'Sports': 1322,
 'Early Access': 2954,
 'Gore': 537,
 'Utilities': 146,
 'Design & Illustration': 87,
 'Web Publishing': 28,
 'Education': 51,
 'Software Training': 31,
 'Sexual Content': 245,
 'Audio Production': 29,
 'Game Development': 17,
 'Photo Editing': 12,
 'Accounting': 6,
 'Documentary': 1,
 'Tutorial': 1}

Some of these items aren't games; some are developer tools or asset packs. I'll focus on popular genres to deliver generalized results. 

In [62]:
genres = list(genre_dict.keys())
genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Animation & Modeling',
 'Video Production',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Nudity',
 'Sports',
 'Early Access',
 'Gore',
 'Utilities',
 'Design & Illustration',
 'Web Publishing',
 'Education',
 'Software Training',
 'Sexual Content',
 'Audio Production',
 'Game Development',
 'Photo Editing',
 'Accounting',
 'Documentary',
 'Tutorial']

In [63]:
genres_to_drop = ['Animation & Modeling', 'Video Production', 'Nudity', 'Gore', 'Utilities', 'Design & Illustration', 'Web Publishing', 'Education', 'Software Training', 'Sexual Content', 'Audio Production', 'Game Development', 'Photo Editing', 'Accounting', 'Documentary', 'Tutorial']
clean_genres = [x for x in genres if x not in genres_to_drop]
clean_genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Sports',
 'Early Access']

In [65]:
print(len(genres))
print(len(genres_to_drop))
print(len(clean_genres))

29
16
13


In [26]:
#content genres pulled using SteamSpy API
steam_df['steamspy_tags']

0              Action;FPS;Multiplayer
1              Action;FPS;Multiplayer
2        FPS;World War II;Multiplayer
3              Action;FPS;Multiplayer
4                   FPS;Action;Sci-fi
                     ...             
27070          Adventure;Indie;Casual
27071          Action;Indie;Adventure
27072             Action;Indie;Casual
27073          Indie;Casual;Adventure
27074          Indie;Casual;Adventure
Name: steamspy_tags, Length: 27075, dtype: object

In [27]:
#content genres
steam_df['steamspy_tags'].value_counts()

Action;Indie;Casual                     845
Action;Adventure;Indie                  714
Early Access;Action;Indie               507
Adventure;Indie;Casual                  442
Indie;Casual                            378
                                       ... 
Action;Sports;Hunting                     1
Stealth;Female Protagonist;Cyberpunk      1
Casual;Space                              1
Horses;Casual;Family Friendly             1
Simulation;Realistic;Naval                1
Name: steamspy_tags, Length: 6423, dtype: int64

In [89]:
steamspy_tags = extract_category(steam_df, 'steamspy_tags')
steamspy_tags

{'Action': 10322,
 'FPS': 405,
 'Multiplayer': 405,
 'World War II': 130,
 'Sci-fi': 157,
 'Classic': 170,
 'Singleplayer': 161,
 'Puzzle': 1116,
 'First-Person': 91,
 'Free to Play': 1662,
 'Zombies': 158,
 'Co-op': 120,
 'MOBA': 13,
 'Strategy': 4173,
 'Shooter': 101,
 'Indie': 16232,
 'Fighting': 117,
 'Cyberpunk': 58,
 'RTS': 174,
 'Hacking': 20,
 'Simulation': 3284,
 'Naval': 39,
 '4X': 37,
 'Turn-Based Strategy': 103,
 'Fantasy': 115,
 'Tanks': 21,
 'Space': 225,
 'RPG': 2785,
 'Animation & Modeling': 54,
 'Video Production': 31,
 'Open World': 243,
 'Competitive': 12,
 'Top-Down Shooter': 27,
 'Casual': 8205,
 'Vampire': 17,
 'Cult Classic': 7,
 'Western': 23,
 'Racing': 765,
 'Destruction': 21,
 'Offroad': 17,
 'Adventure': 7770,
 'Match 3': 119,
 'Arcade': 203,
 'Hidden Object': 460,
 'Tower Defense': 168,
 'Sniper': 8,
 'Stealth': 93,
 'Base-Building': 26,
 'Villain Protagonist': 9,
 'Story Rich': 159,
 'Female Protagonist': 199,
 'Platformer': 648,
 'Comedy': 89,
 'Turn-Base

In [102]:
len(steamspy_tags)

339

In [101]:
for tag in sorted(steamspy_tags, key=steamspy_tags.get, reverse=True):
    print(tag, steamspy_tags[tag])
    
#maybe remove counts below 30, if you're really into those categories, you can browse those yourself

Indie 16232
Action 10322
Casual 8205
Adventure 7770
Strategy 4173
Simulation 3284
Early Access 2967
RPG 2785
Free to Play 1662
Puzzle 1116
VR 961
Sports 781
Racing 765
Platformer 648
Nudity 570
Sexual Content 557
Visual Novel 546
Violent 545
Anime 538
Horror 493
Point & Click 492
Gore 476
Hidden Object 460
FPS 405
Multiplayer 405
Massively Multiplayer 377
Pixel Graphics 249
Shoot 'Em Up 245
Open World 243
Survival 235
Space 225
Arcade 203
Female Protagonist 199
RPGMaker 192
RTS 174
Turn-Based 171
Classic 170
Tower Defense 168
Card Game 164
2D 163
Singleplayer 161
Story Rich 159
Zombies 158
Sci-fi 157
Utilities 138
World War II 130
Great Soundtrack 125
Co-op 120
Match 3 119
Fighting 117
Management 116
Fantasy 115
Retro 114
Music 105
Board Game 105
Rogue-like 104
Turn-Based Strategy 103
Shooter 101
Bullet Hell 96
Sandbox 94
JRPG 94
Stealth 93
First-Person 91
Comedy 89
Family Friendly 86
Atmospheric 85
Difficult 85
Education 84
Hack and Slash 83
Local Multiplayer 80
Design & Illustration 

In [66]:
#number of achievements available
steam_df['achievements']

0         0
1         0
2         0
3         0
4         0
         ..
27070     7
27071     0
27072    24
27073     0
27074     0
Name: achievements, Length: 27075, dtype: int64

In [67]:
steam_df['achievements'].value_counts()

0       11864
10        679
12        618
20        574
15        490
        ...  
184         1
168         1
1487        1
319         1
4094        1
Name: achievements, Length: 410, dtype: int64

In [30]:
#is achievements a meaningful consideration for buyers?
steam_df[steam_df['achievements']==4094]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
13740,604490,Running Through Russia,2017-05-21,1,Spell Helix,Spell Helix,windows;mac;linux,0,Single-player;Steam Achievements,Adventure;Casual;Indie;Early Access,Early Access;Indie;Casual,4094,751,335,22,22,50000-100000,0.79


In [68]:
steam_df['positive_ratings']

0        124534
1          3318
2          3416
3          1273
4          5250
          ...  
27070         3
27071         8
27072         0
27073         2
27074         4
Name: positive_ratings, Length: 27075, dtype: int64

In [69]:
steam_df['negative_ratings']

0        3339
1         633
2         398
3         267
4         288
         ... 
27070       0
27071       1
27072       1
27073       0
27074       0
Name: negative_ratings, Length: 27075, dtype: int64

In [70]:
no_ratings = steam_df[ (steam_df['positive_ratings'] == 0) &  (steam_df['negative_ratings'] == 0) ]
no_ratings
#every game in this dataset has at least 1 rating
#there shouldn't be any divide-by-0 errors if I create a % positive/% negative ratings feature

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price


In [72]:
steam_df['total_ratings'] = steam_df['positive_ratings'] + steam_df['negative_ratings']
steam_df['total_ratings']

0        127873
1          3951
2          3814
3          1540
4          5538
          ...  
27070         3
27071         9
27072         1
27073         2
27074         4
Name: total_ratings, Length: 27075, dtype: int64

In [73]:
steam_df['percent_positive'] = steam_df['positive_ratings'] / steam_df['total_ratings']
steam_df['percent_positive']

0        0.973888
1        0.839787
2        0.895648
3        0.826623
4        0.947996
           ...   
27070    1.000000
27071    0.888889
27072    0.000000
27073    1.000000
27074    1.000000
Name: percent_positive, Length: 27075, dtype: float64

In [40]:
#average user playtime (minutes)
steam_df['average_playtime']

0        17612
1          277
2          187
3          258
4          624
         ...  
27070        0
27071        0
27072        0
27073        0
27074        0
Name: average_playtime, Length: 27075, dtype: int64

In [41]:
steam_df[steam_df['average_playtime'] == 0]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
26,1002,Rag Doll Kung Fu,2005-10-12,1,Mark Healey,Mark Healey,windows,0,Single-player;Multi-player,Indie,Indie;Fighting;Multiplayer,0,40,17,0,0,20000-50000,5.99
29,1300,SiN Episodes: Emergence,2006-05-10,1,Ritual Entertainment,Ritual Entertainment,windows,0,Single-player;Stats,Action,Action;FPS;Cyberpunk,0,468,61,0,0,100000-200000,7.19
34,1600,Dangerous Waters,2006-02-07,1,Sonalysts,Strategy First,windows,0,Single-player;Multi-player,Strategy,Strategy;Simulation;Naval,0,140,44,0,0,50000-100000,22.99
35,1610,Space Empires IV Deluxe,2006-02-07,1,Malfador Machinations,Strategy First,windows,0,Single-player;Multi-player,Strategy,Strategy;4X;Sci-fi,0,112,26,0,0,50000-100000,6.99
36,1630,Disciples II: Rise of the Elves,2006-07-06,1,Strategy First,Strategy First,windows,0,Single-player;Multi-player;Co-op,Strategy,Strategy;Turn-Based Strategy;Fantasy,0,451,108,0,0,100000-200000,4.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19


In [51]:
no_playtime = steam_df[steam_df['average_playtime'] == 0]
no_playtime.sort_values(by=['owners'], ascending=False)
no_playtime.shape

(20905, 18)

In [7]:
steam_df['median_playtime'].value_counts()

0       20905
1         155
3          72
2          52
9          48
        ...  
636         1
684         1
4164        1
748         1
6061        1
Name: median_playtime, Length: 1312, dtype: int64

In [8]:
#number of users who own this game, given as a range
steam_df['owners']

0        10000000-20000000
1         5000000-10000000
2         5000000-10000000
3         5000000-10000000
4         5000000-10000000
               ...        
27070              0-20000
27071              0-20000
27072              0-20000
27073              0-20000
27074              0-20000
Name: owners, Length: 27075, dtype: object

In [59]:
#price of game in USD
steam_df['price']

0        7.19
1        3.99
2        3.99
3        3.99
4        3.99
         ... 
27070    2.09
27071    1.69
27072    3.99
27073    5.19
27074    5.19
Name: price, Length: 27075, dtype: float64

In [60]:
steam_df['price'].value_counts()

3.99     3211
0.79     2892
0.00     2560
6.99     2050
7.19     1304
         ... 
7.42        1
3.03        1
6.10        1
11.75       1
4.70        1
Name: price, Length: 282, dtype: int64

In [74]:
steam_df.iloc[0]

appid                                                              10
name                                                   Counter-Strike
release_date                                               2000-11-01
english                                                             1
developer                                                       Valve
publisher                                                       Valve
platforms                                           windows;mac;linux
required_age                                                        0
categories          Multi-player;Online Multi-Player;Local Multi-P...
genres                                                         Action
steamspy_tags                                  Action;FPS;Multiplayer
achievements                                                        0
positive_ratings                                               124534
negative_ratings                                                 3339
average_playtime    

## Dataframe with Encoded Values

In [114]:
genres_df = steam_df[['appid', 'name', 'genres']]
genres_df

,appid,name,genres
0,10,Counter-Strike,Action
1,20,Team Fortress Classic,Action
2,30,Day of Defeat,Action
3,40,Deathmatch Classic,Action
4,50,Half-Life: Opposing Force,Action
...,...,...,...
27070,1065230,Room of Pandora,Adventure;Casual;Indie
27071,1065570,Cyber Gun,Action;Adventure;Indie
27072,1065650,Super Star Blast,Action;Casual;Indie
27073,1066700,New Yankee 7: Deer Hunters,Adventure;Casual;Indie


In [117]:
genres_df['genres_split'] = genres_df['genres'].str.split(';')
genres_df

<ipython-input-117-6a02c8903761>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_df['genres_split'] = genres_df['genres'].str.split(';')


,appid,name,genres,genres_split
0,10,Counter-Strike,Action,[Action]
1,20,Team Fortress Classic,Action,[Action]
2,30,Day of Defeat,Action,[Action]
3,40,Deathmatch Classic,Action,[Action]
4,50,Half-Life: Opposing Force,Action,[Action]
...,...,...,...,...
27070,1065230,Room of Pandora,Adventure;Casual;Indie,"[Adventure, Casual, Indie]"
27071,1065570,Cyber Gun,Action;Adventure;Indie,"[Action, Adventure, Indie]"
27072,1065650,Super Star Blast,Action;Casual;Indie,"[Action, Casual, Indie]"
27073,1066700,New Yankee 7: Deer Hunters,Adventure;Casual;Indie,"[Adventure, Casual, Indie]"


In [123]:
mlb = MultiLabelBinarizer()
genres_mlb = mlb.fit_transform(genres_df['genres_split'])

In [125]:
list(mlb.classes_)

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Documentary',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Gore',
 'Indie',
 'Massively Multiplayer',
 'Nudity',
 'Photo Editing',
 'RPG',
 'Racing',
 'Sexual Content',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Tutorial',
 'Utilities',
 'Video Production',
 'Violent',
 'Web Publishing']

In [127]:
split_genres_df = pd.DataFrame(genres_mlb, genres_df.index, mlb.classes_)
split_genres_df

,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Documentary,Early Access,Education,...,Sexual Content,Simulation,Software Training,Sports,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27071,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27072,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27073,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
ohe = OneHotEncoder(handle_unknown='ignore')
genres_ohe = ohe.fit_transform(genres_df[['genres']])

In [88]:
clean_genres

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Simulation',
 'Racing',
 'Violent',
 'Massively Multiplayer',
 'Sports',
 'Early Access']

In [87]:
genres_ohe

<27075x1552 sparse matrix of type '<class 'numpy.float64'>'
	with 27075 stored elements in Compressed Sparse Row format>

In [92]:
len(steamspy_tags.keys())

339